# Quick Start: Azure AI Foundry SDK

### 01 Create a new agent
##### Azure AI Foundry Agent Service allows you to create AI agents tailored to your needs through custom instructions and augmented by advanced tools like code interpreter, and custom functions.

In [1]:
# --- Standard Library Imports ---
import os
import sys
import pprint

# --- Azure AI Foundry SDK for AI agents and services ---
from azure.ai.agents.models import CodeInterpreterTool, FilePurpose, FileSearchTool, ListSortOrder
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv()

pprint.pp(sys.executable)

('c:\\Users\\Yu-Hong (Frank) '
 'Lin\\Repos\\azure-ai-agent\\.venv\\Scripts\\python.exe')


In [ ]:
# You need to login to Azure subscription via Azure CLI first

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],  # Project endpoint from environment variable
    credential=DefaultAzureCredential(),      # Use Azure Default Credential for authentication
)

code_interpreter = CodeInterpreterTool()
with project_client:
    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-agent",                            # Name of the agent
        instructions="You are a helpful agent",     # Instructions for the agent
        tools=code_interpreter.definitions,         # Attach the tool
    )
    print(f"Created agent, ID: {agent.id}")

    # Create a thread for communication
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")
    
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",                                      # Role of the message sender
        content="What is the weather in Seattle today?",  # Message content
    )
    print(f"Created message, ID: {message['id']}")
    
    # Create and process an agent run
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")
    
    # Check if the run failed
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    
    # Fetch and log all messages
    messages = project_client.agents.messages.list(thread_id=thread.id)
    for message in messages:
        print(f"Role: {message.role}, Content: {message.content}")

Created agent, ID: asst_NOGmu371G90oELLj2DbXOlMa
Created thread, ID: thread_6o4mHMTq7B2U2xdUViMWt80C
Created message, ID: msg_dO3o2JwaRScBBlkuZbtKHIgg
Run finished with status: RunStatus.COMPLETED
Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': "I don't have current internet access, so I cannot retrieve live weather updates for Seattle. However, you can check the weather using services like Google Weather, AccuWeather, or a weather app on your device for an up-to-date forecast.\n\nIf you'd like general climate info about Seattle or help understanding a forecast, feel free to ask!", 'annotations': []}}]
Role: MessageRole.USER, Content: [{'type': 'text', 'text': {'value': 'What is the weather in Seattle today?', 'annotations': []}}]


### 02 Listing AI Project Agents

In [ ]:
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),  
)

# List the agents
with project_client:
    
    agent_list = project_client.agents.list_agents()
    for _agent in agent_list:
        print(_agent)

{'id': 'asst_NOGmu371G90oELLj2DbXOlMa', 'object': 'assistant', 'created_at': 1752763579, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_jASK1GS6htj8mrQZnBDAb7nw', 'object': 'assistant', 'created_at': 1752763369, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_lXR1U4CTAZRGiR1Xc7dm7ZPu', 'object': 'assistant', 'created_at': 1752763201, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.

### 03 Deleting an Agent from AI Project Client

In [ ]:
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),  
)

# Delete the agent when done
with project_client:
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

---

##### Reference: [Create a new Azure AI Foundry Agent Service project - Azure AI Foundry | Microsoft Learn](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/quickstart?pivots=programming-language-python-azure)